In [ ]:
#This script is used to perform dynamic FBA with Y. lipolytica.
#Author: Martin KavÅ¡?ek; martin.kavscek@uni-graz.at
#date: 25.9.2015

from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite


model = read_sbml_model('Kavscek2015_mod.xml');
#dFBA calculation
#standard conditions with glucose uptake

model.constraints (model, 'EX_co2(e)', -1000, 'l');
model = changeRxnBounds (model, 'EX_h2o(e)', -1000, 'l');
model = changeRxnBounds (model, 'EX_h(e)', -1000, 'l');
model = changeRxnBounds (model, 'EX_inost(e)', -1000, 'l');
model = changeRxnBounds (model, 'EX_k(e)', -1000, 'l');
model = changeRxnBounds (model, 'EX_na1(e)', -1000, 'l');
model = changeRxnBounds (model, 'EX_nh4(e)', -1000, 'l');
model = changeRxnBounds (model, 'EX_o2(e)', -1000, 'l');
model = changeRxnBounds (model, 'EX_pi(e)', -1000, 'l');
model = changeRxnBounds (model, 'EX_so4(e)', -1000, 'l');
model = changeRxnBounds (model, 'EX_glc(e)', -4.0, 'l');
model = changeRxnBounds (model, 'EX_glc(e)', 1000, 'u');
model = changeRxnBounds (model, 'EX_glyc(e)', 0, 'l');
model = changeRxnBounds (model, 'EX_glyc(e)', 1000, 'u');
uptakeRxns = printUptakeBound(model)

model = changeObjective (model, 'biomass_013',1);
mantainance = 4 * 5
%mantainannce = 8.78 * 2.28


%#config fur function  dynamicFBA(model,substrateRxns,initConcentrations,initBiomass,timeStep,nSteps,plotRxns)
substrateRxns = 'EX_glc(e)';
initConcentrations = 111; # 20g/L
initBiomass = 0.03;
timeStep = 0.5;
nSteps = 40;
plotRxns = {'EX_glc(e)'};
[concentrationMatrix,excRxnNames,timeVec,biomassVec] =  dynamicFBA(model,substrateRxns,initConcentrations,initBiomass,timeStep,nSteps,plotRxns);


model = changeRxnBounds (model, 'ATPM', mantainance, 'b');
GR = optimizeCbModel(model, 'max', 'one')


%Citric acid production  
%standard conditions with glucose uptake

model = changeRxnBounds (model, 'EX_glc(e)', -0.350, 'l'); %calculated only for stationary phase
uptakeRxns = printUptakeBound(model);


mantainance1 = 0.35*5.3;
model = changeRxnBounds (model, 'ATPM', mantainance1, 'b');


model = changeObjective (model, 'LDparticle_SC_e_tp'); %lipid production


LIPIDwoCIT = optimizeCbModel(model, 'max', 'one') %lipid production without citrate excretion
mantainance2 = 0.35*4.75;
model = changeRxnBounds (model, 'ATPM', mantainance2, 'b');

LIPIDwoCIT = optimizeCbModel(model, 'max', 'one') %lipid production without citrate excretion

CAexcretion = 3.339;
CAmol = 192.194;
duration = 29;
biomass = 3.0;
CAkonc = (CAexcretion / CAmol) * 1000; %conc of CA in mmoles

CArate = (CAkonc / biomass) / duration;
model = changeRxnBounds (model, 'EX_cit(e)', CArate, 'b');
LIPIDCIT = optimizeCbModel(model, 'max', 'one') %lipid production with citrate excretion
diff = LIPIDCIT.f/LIPIDWT.f